# Imports

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf

from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, concatenate, Input, BatchNormalization
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Connect drive

In [2]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


# Dataset

In [3]:
path_images="/content/drive/My Drive/uc project/images.npz"
dict_image_data = np.load(path_images)
image_data = dict_image_data['arr_0']

In [4]:
path_cities="/content/drive/My Drive/uc project/cities.npz"
dict_city_data = np.load(path_cities)
city_data = dict_city_data['arr_0']

path_class="/content/drive/My Drive/uc project/classes.npz"
dict_class_data = np.load(path_class)
class_data = dict_class_data['arr_0']

In [5]:
# all 512 images resized to 128
# data_512 = image_data[:,0][indices]
data_512 = image_data[:,0]
# all 512 images cropped to 256 images and resized to 128
# data_256 = image_data[:,1][indices]
data_256 = image_data[:,1]
# all 512 images cropped to 128 images
# data_128 = image_data[:,2][indices]
data_128 = image_data[:,2]

In [6]:
city_data.shape, data_512.shape, data_512.shape

((28195,), (28195, 128, 128, 3), (28195, 128, 128, 3))

Tranforming labels to onehot encoding

In [7]:
from sklearn.preprocessing import LabelBinarizer
city_label_binarizer = LabelBinarizer()
city_labels = city_label_binarizer.fit_transform(city_data)
class_label_binarizer = LabelBinarizer()
class_labels = class_label_binarizer.fit_transform(class_data)

Split train and test data


In [8]:
train_512, test_512, train_256, test_256, train_128, test_128, train_city, test_city, train_class, test_class = train_test_split(data_512, data_256, data_128, city_labels, class_labels, test_size=0.2, random_state=42)

In [9]:
from collections import Counter

Counter(np.argmax(train_city, axis=1))

Counter({0: 4031, 1: 4340, 3: 6313, 2: 3678, 4: 4194})

In [10]:
city_label_binarizer.classes_

array(['athina, gr', 'berlin, de', 'budapest, hu', 'madrid, es',
       'roma, it'], dtype='<U12')

In [11]:
class_label_binarizer.classes_

array(['Agricultural + Semi-natural areas + Wetlands', 'Airports',
       'Construction sites', 'Continuous Urban Fabric (S.L.  80%)',
       'Discontinuous Dense Urban Fabric (S.L.  50% -  80%)',
       'Discontinuous Low Density Urban Fabric (S.L.  10% - 30%)',
       'Discontinuous Medium Density Urban Fabric (S.L.  30% - 50%)',
       'Discontinuous Very Low Density Urban Fabric (S.L.  10%)',
       'Fast transit roads and associated land', 'Forests',
       'Green urban areas',
       'Industrial, commercial, public, military and private units',
       'Mineral extraction and dump sites',
       'Other roads and associated land', 'Port areas',
       'Railways and associated land', 'Sports and leisure facilities',
       'Water bodies'], dtype='<U59')

In [12]:
Counter(np.argmax(test_city, axis=1))

Counter({2: 980, 1: 1126, 0: 991, 3: 1527, 4: 1015})

In [13]:
Counter(np.argmax(train_class, axis=1))

Counter({3: 1446,
         9: 909,
         0: 6836,
         4: 1520,
         13: 7428,
         11: 1185,
         10: 650,
         6: 608,
         17: 369,
         14: 117,
         16: 362,
         2: 99,
         15: 373,
         8: 255,
         5: 205,
         1: 132,
         7: 37,
         12: 25})

In [14]:
Counter(np.argmax(test_class, axis=1))

Counter({13: 1870,
         11: 338,
         4: 391,
         10: 171,
         9: 225,
         2: 28,
         0: 1681,
         6: 139,
         3: 332,
         14: 30,
         5: 55,
         17: 100,
         16: 89,
         15: 91,
         8: 54,
         1: 35,
         7: 7,
         12: 3})

# Networks

In [15]:
def basic_model(input_name):
  # 1
  model_input = Input(shape=(128, 128, 3), name=input_name)
  conv2d1 = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(model_input)
  norm1 = BatchNormalization()(conv2d1)
  # 2
  pool1 = MaxPooling2D(pool_size=(2,2))(norm1)
  norm2 = BatchNormalization()(pool1)
  # 1
  conv2d2 = Conv2D(filters=32, kernel_size=(3,3), activation='relu')(norm2)
  norm3 = BatchNormalization()(conv2d2)
  # 2
  pool2 = MaxPooling2D(pool_size=(2,2))(norm3)
  norm4 = BatchNormalization()(pool2)
  # 15
  flat = Flatten()(norm4)
  dense = Dense(2048, activation='relu')(flat)
  model_output = Dropout(0.2)(dense)
  return model_input, model_output

## e128

Get input and initial output layers

In [16]:
basic_input, basic_output = basic_model('main_input')

Attach 2 outputs

In [17]:
#City
merged_hidden_city = Dense(2048, activation='relu')(basic_output)
merged_drop_city = Dropout(0.2)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_drop_city)

#Class
merged_hidden_class = Dense(2048, activation='relu')(basic_output)
merged_drop_class = Dropout(0.2)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_drop_class)

In [18]:
e128_model = Model(inputs=basic_input, outputs=[city_output, class_output])
# e128_model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])
gamma = 0.5
e128_model.compile(optimizer='adam',
                  loss={'city_output': 'categorical_crossentropy', 
                        'class_output': 'categorical_crossentropy'},
                  loss_weights={'city_output': gamma, 
                                'class_output': 1 - gamma}, 
                  metrics=['accuracy'])

In [19]:
e128_history = e128_model.fit(train_128, {'city_output': train_city, 'class_output': train_class},
                              # batch_size=20,
                              epochs=15,
                              shuffle=True,
                              validation_data=(test_128, {'city_output': test_city, 'class_output': test_class})
                 )

Epoch 1/15
705/705 [==============================] - 33s 34ms/step - loss: 1.7657 - city_output_loss: 1.2703 - class_output_loss: 2.2610 - city_output_accuracy: 0.6649 - class_output_accuracy: 0.4085 - val_loss: 1.4174 - val_city_output_loss: 1.0693 - val_class_output_loss: 1.7654 - val_city_output_accuracy: 0.5973 - val_class_output_accuracy: 0.4029
Epoch 2/15
705/705 [==============================] - 23s 33ms/step - loss: 1.1041 - city_output_loss: 0.6269 - class_output_loss: 1.5812 - city_output_accuracy: 0.7746 - class_output_accuracy: 0.4577 - val_loss: 1.3575 - val_city_output_loss: 0.9523 - val_class_output_loss: 1.7628 - val_city_output_accuracy: 0.6203 - val_class_output_accuracy: 0.4362
Epoch 3/15
705/705 [==============================] - 23s 33ms/step - loss: 0.9858 - city_output_loss: 0.5337 - class_output_loss: 1.4380 - city_output_accuracy: 0.8154 - class_output_accuracy: 0.5098 - val_loss: 1.6667 - val_city_output_loss: 1.2949 - val_class_output_loss: 2.0385 - val_cit

## e256

Get input and initial output layers

In [20]:
basic_input, basic_output = basic_model('main_input')

Attach 2 outputs

In [21]:
#City
merged_hidden_city = Dense(2048, activation='relu')(basic_output)
merged_drop_city = Dropout(0.2)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_drop_city)

#Class
merged_hidden_class = Dense(2048, activation='relu')(basic_output)
merged_drop_class = Dropout(0.2)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_drop_class)

In [22]:
e256_model = Model(inputs=basic_input, outputs=[city_output, class_output])
# e256_model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])
gamma = 0.5
e256_model.compile(optimizer='adam',
                  loss={'city_output': 'categorical_crossentropy', 
                        'class_output': 'categorical_crossentropy'},
                  loss_weights={'city_output': gamma, 
                                'class_output': 1 - gamma}, 
                  metrics=['accuracy'])

In [23]:
e256_history = e256_model.fit(train_256, {'city_output': train_city, 'class_output': train_class},
                              # batch_size=20,
                              epochs=15,
                              shuffle=True,
                              validation_data=(test_256, {'city_output': test_city, 'class_output': test_class})
                 )

Epoch 1/15
705/705 [==============================] - 25s 33ms/step - loss: 1.6453 - city_output_loss: 1.1674 - class_output_loss: 2.1232 - city_output_accuracy: 0.7136 - class_output_accuracy: 0.4238 - val_loss: 1.9798 - val_city_output_loss: 2.0517 - val_class_output_loss: 1.9079 - val_city_output_accuracy: 0.3105 - val_class_output_accuracy: 0.3910
Epoch 2/15
705/705 [==============================] - 23s 33ms/step - loss: 1.0105 - city_output_loss: 0.5318 - class_output_loss: 1.4892 - city_output_accuracy: 0.8194 - class_output_accuracy: 0.4847 - val_loss: 2.3825 - val_city_output_loss: 2.6804 - val_class_output_loss: 2.0845 - val_city_output_accuracy: 0.4660 - val_class_output_accuracy: 0.3878
Epoch 3/15
705/705 [==============================] - 23s 33ms/step - loss: 0.8613 - city_output_loss: 0.4385 - class_output_loss: 1.2842 - city_output_accuracy: 0.8556 - class_output_accuracy: 0.5562 - val_loss: 1.6923 - val_city_output_loss: 1.6038 - val_class_output_loss: 1.7809 - val_cit

## e512

Get input and initial output layers

In [24]:
basic_input, basic_output = basic_model('main_input')

Attach 2 outputs

In [25]:
#City
merged_hidden_city = Dense(2048, activation='relu')(basic_output)
merged_drop_city = Dropout(0.2)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_drop_city)

#Class
merged_hidden_class = Dense(2048, activation='relu')(basic_output)
merged_drop_class = Dropout(0.2)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_drop_class)

In [26]:
e512_model = Model(inputs=basic_input, outputs=[city_output, class_output])
# e512_model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])
gamma = 0.5
e512_model.compile(optimizer='adam',
                  loss={'city_output': 'categorical_crossentropy', 
                        'class_output': 'categorical_crossentropy'},
                  loss_weights={'city_output': gamma, 
                                'class_output': 1 - gamma}, 
                  metrics=['accuracy'])

In [27]:
e512_history = e512_model.fit(train_512, {'city_output': train_city, 'class_output': train_class},
                              # batch_size=20,
                              epochs=15,
                              shuffle=True,
                              validation_data=(test_512, {'city_output': test_city, 'class_output': test_class})
                 )

Epoch 1/15
705/705 [==============================] - 25s 34ms/step - loss: 1.6820 - city_output_loss: 1.1715 - class_output_loss: 2.1926 - city_output_accuracy: 0.7384 - class_output_accuracy: 0.4311 - val_loss: 1.5223 - val_city_output_loss: 0.8687 - val_class_output_loss: 2.1758 - val_city_output_accuracy: 0.7156 - val_class_output_accuracy: 0.3836
Epoch 2/15
705/705 [==============================] - 23s 33ms/step - loss: 0.9127 - city_output_loss: 0.4036 - class_output_loss: 1.4218 - city_output_accuracy: 0.8638 - class_output_accuracy: 0.5020 - val_loss: 2.0241 - val_city_output_loss: 2.2222 - val_class_output_loss: 1.8259 - val_city_output_accuracy: 0.6146 - val_class_output_accuracy: 0.3916
Epoch 3/15
705/705 [==============================] - 23s 33ms/step - loss: 0.7724 - city_output_loss: 0.3443 - class_output_loss: 1.2004 - city_output_accuracy: 0.8910 - class_output_accuracy: 0.5792 - val_loss: 1.4901 - val_city_output_loss: 1.1038 - val_class_output_loss: 1.8764 - val_cit

## final

Three inputs
  1. e128
  1. e256
  1. e512

Two outputs
  1. city
  1. class

Create the three basic input models

In [28]:
e128_input, e128_output = basic_model('e128_input')
e256_input, e256_output = basic_model('e256_input')
e512_input, e512_output = basic_model('e512_input')

Concatenating outputs of three models

In [29]:
merged = concatenate([e128_output, e256_output, e512_output])

Defining the outputs

In [30]:
#City
merged_hidden_city = Dense(2048, activation='relu')(merged)
merged_drop_city = Dropout(0.2)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_drop_city)

#Class
merged_hidden_class = Dense(2048, activation='relu')(merged)
merged_drop_class = Dropout(0.2)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_drop_class)

In [31]:
final_model = Model(inputs=[e128_input, e256_input, e512_input], outputs=[city_output, class_output])
# final_model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'], metrics=['accuracy'])
gamma = 0.5
final_model.compile(optimizer='adam',
                  loss={'city_output': 'categorical_crossentropy', 
                        'class_output': 'categorical_crossentropy'},
                  loss_weights={'city_output': gamma, 
                                'class_output': 1 - gamma}, 
                  metrics=['accuracy'])

Training model

In [32]:
final_history = final_model.fit({'e128_input': train_128, 'e256_input': train_256, 'e512_input': train_512}, {'city_output': train_city, 'class_output': train_class},
                                # batch_size=20,
                                epochs=15,
                                shuffle=True,
                                validation_data=({'e128_input': test_128, 'e256_input': test_256, 'e512_input': test_512}, {'city_output': test_city, 'class_output': test_class})
                 )
                

Epoch 1/15
705/705 [==============================] - 68s 94ms/step - loss: 1.9144 - city_output_loss: 1.4045 - class_output_loss: 2.4244 - city_output_accuracy: 0.7386 - class_output_accuracy: 0.4114 - val_loss: 1.2435 - val_city_output_loss: 0.7855 - val_class_output_loss: 1.7016 - val_city_output_accuracy: 0.7526 - val_class_output_accuracy: 0.4217
Epoch 2/15
705/705 [==============================] - 65s 92ms/step - loss: 1.0158 - city_output_loss: 0.4582 - class_output_loss: 1.5735 - city_output_accuracy: 0.8456 - class_output_accuracy: 0.4599 - val_loss: 1.4373 - val_city_output_loss: 0.8218 - val_class_output_loss: 2.0528 - val_city_output_accuracy: 0.7452 - val_class_output_accuracy: 0.4354
Epoch 3/15
705/705 [==============================] - 65s 92ms/step - loss: 0.8956 - city_output_loss: 0.3373 - class_output_loss: 1.4540 - city_output_accuracy: 0.8877 - class_output_accuracy: 0.4888 - val_loss: 1.1409 - val_city_output_loss: 0.6598 - val_class_output_loss: 1.6219 - val_cit

## Evaluate


In [33]:
y_e128_city_pred, y_e128_class_pred = e128_model.predict(test_128)
y_e256_city_pred, y_e256_class_pred = e256_model.predict(test_256)
y_e512_city_pred, y_e512_class_pred = e512_model.predict(test_512)
y_final_city_pred, y_final_class_pred = final_model.predict({'e128_input': test_128, 'e256_input': test_256, 'e512_input': test_512})

177/177 [==============================] - 3s 13ms/step


In [34]:
y_e128_city_pred = np.argmax(y_e128_city_pred, axis=1)
y_e256_city_pred = np.argmax(y_e256_city_pred, axis=1)
y_e512_city_pred = np.argmax(y_e512_city_pred, axis=1)
y_final_city_pred = np.argmax(y_final_city_pred, axis=1)

y_e128_class_pred = np.argmax(y_e128_class_pred, axis=1)
y_e256_class_pred = np.argmax(y_e256_class_pred, axis=1)
y_e512_class_pred = np.argmax(y_e512_class_pred, axis=1)
y_final_class_pred = np.argmax(y_final_class_pred, axis=1)

In [35]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score


city_y = np.argmax(test_city, axis=1)
class_y = np.argmax(test_class, axis=1)

e128_city_cm = confusion_matrix(city_y, y_e128_city_pred)
e256_city_cm = confusion_matrix(city_y, y_e256_city_pred)
e512_city_cm = confusion_matrix(city_y, y_e512_city_pred)
final_city_cm = confusion_matrix(city_y, y_final_city_pred)

e128_class_cm = confusion_matrix(class_y, y_e128_class_pred)
e256_class_cm = confusion_matrix(class_y, y_e256_class_pred)
e512_class_cm = confusion_matrix(class_y, y_e512_class_pred)
final_class_cm = confusion_matrix(class_y, y_final_class_pred)

e128_city_f1 = f1_score(city_y, y_e128_city_pred, average='micro')
e256_city_f1 = f1_score(city_y, y_e256_city_pred, average='micro')
e512_city_f1 = f1_score(city_y, y_e512_city_pred, average='micro')
final_city_f1 = f1_score(city_y, y_final_city_pred, average='micro')

e128_class_f1 = f1_score(class_y, y_e128_class_pred, average='micro')
e256_class_f1 = f1_score(class_y, y_e256_class_pred, average='micro')
e512_class_f1 = f1_score(class_y, y_e512_class_pred, average='micro')
final_class_f1 = f1_score(class_y, y_final_class_pred, average='micro')

In [36]:
e128_city_cm

array([[ 707,    7,    8,  213,   56],
       [  99,  818,    6,  105,   98],
       [ 159,   91,  434,  165,  131],
       [  91,   46,    5, 1314,   71],
       [ 198,   57,    9,  322,  429]])

In [37]:
e256_city_cm

array([[ 506,   35,    9,  395,   46],
       [  67,  928,   17,   83,   31],
       [  66,  120,  330,  427,   37],
       [  22,   15,   11, 1451,   28],
       [ 190,   55,   19,  437,  314]])

In [38]:
e512_city_cm

array([[721, 124,  30,  19,  97],
       [ 10, 543, 536,   0,  37],
       [157,  32, 661,  59,  71],
       [ 50, 216, 169, 930, 162],
       [ 73, 324, 126,  10, 482]])

In [39]:
final_city_cm

array([[ 789,   11,  142,   17,   32],
       [  40,  922,  111,   22,   31],
       [   4,   17,  940,    9,   10],
       [  39,   65,  146, 1240,   37],
       [  48,   59,  248,   42,  618]])

In [40]:
e128_class_cm

array([[1166,    1,   12,   46,   28,    3,    3,    0,    1,   22,   11,
          13,    4,  347,    0,    0,    3,   21],
       [  11,    0,    0,    4,    0,    0,    0,    0,    0,    0,    0,
           3,    0,   17,    0,    0,    0,    0],
       [   9,    0,    0,    4,    0,    0,    1,    1,    0,    0,    0,
           0,    0,   13,    0,    0,    0,    0],
       [  20,    0,    4,  137,   21,    2,    2,    1,    0,    0,    2,
           7,    0,  136,    0,    0,    0,    0],
       [  37,    1,    8,   90,   29,    6,    7,    1,    0,    3,    4,
          15,    0,  180,    0,    1,    5,    4],
       [   8,    0,    3,    6,    1,    4,    2,    0,    0,    0,    1,
           0,    0,   30,    0,    0,    0,    0],
       [  24,    0,    1,   24,   12,    3,    3,    1,    0,    1,    1,
           8,    0,   60,    0,    0,    1,    0],
       [   5,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    2,    0,    0,    0,    0],


In [41]:
e256_class_cm

array([[588,   0,   0, 184, 100,  33, 249,   0,   0,   4,  15,   2,   0,
        486,   1,   3,   2,  14],
       [  7,   0,   0,   3,   4,   0,   6,   0,   0,   0,   0,   3,   0,
         10,   0,   0,   0,   2],
       [  6,   0,   0,   3,   4,   2,   3,   0,   0,   0,   0,   0,   0,
         10,   0,   0,   0,   0],
       [  3,   0,   0,  96,  60,   5,  46,   0,   0,   0,   0,   1,   0,
        120,   0,   1,   0,   0],
       [  4,   0,   0,  61, 122,  15,  88,   0,   0,   0,   0,   0,   0,
        100,   0,   0,   1,   0],
       [  4,   0,   0,   3,   6,   5,  12,   0,   0,   0,   2,   0,   0,
         23,   0,   0,   0,   0],
       [  1,   0,   0,   8,  38,   7,  32,   0,   0,   0,   1,   0,   0,
         49,   0,   2,   1,   0],
       [  0,   0,   0,   1,   1,   1,   2,   0,   0,   0,   0,   0,   0,
          1,   0,   1,   0,   0],
       [  8,   0,   0,  11,   1,   0,  12,   0,   0,   0,   1,   0,   0,
         20,   0,   0,   0,   1],
       [ 19,   0,   0,  12,  12,   4,

In [42]:
e512_class_cm

array([[ 676,    0,    1,    8,   15,    0,    2,    0,    0,    2,    5,
           1,    0,  958,    0,    3,    8,    2],
       [  13,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,
           1,    0,   20,    0,    0,    0,    0],
       [  12,    0,    0,    1,    3,    0,    0,    0,    0,    0,    1,
           2,    0,    8,    0,    1,    0,    0],
       [  18,    0,    0,   45,   35,    0,    5,    0,    0,    0,    1,
           7,    0,  214,    0,    2,    4,    1],
       [  16,    0,    0,   16,   75,    0,    7,    0,    0,    0,    2,
           9,    0,  261,    0,    1,    2,    2],
       [  12,    0,    0,    2,    7,    0,    2,    0,    0,    0,    1,
           0,    0,   29,    0,    0,    1,    1],
       [   9,    0,    0,    6,   23,    0,    4,    0,    0,    0,    0,
           2,    0,   89,    0,    3,    1,    2],
       [   1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    6,    0,    0,    0,    0],


In [43]:
final_class_cm

array([[1196,    3,    0,    0,   18,    0,    1,    0,    4,   80,    4,
          15,    0,  341,    0,    4,    1,   14],
       [  25,    0,    0,    0,    0,    0,    0,    0,    1,    2,    0,
           0,    0,    7,    0,    0,    0,    0],
       [  14,    0,    0,    1,    0,    0,    0,    0,    0,    0,    0,
           1,    0,   12,    0,    0,    0,    0],
       [   6,    0,    0,   66,   65,    1,    0,    0,    0,    1,    1,
          13,    0,  178,    0,    0,    0,    1],
       [  19,    0,    0,   27,  123,    0,   12,    0,    1,    7,    1,
          25,    0,  168,    0,    0,    7,    1],
       [  15,    0,    0,    1,    3,    1,    0,    0,    0,    6,    0,
           1,    0,   28,    0,    0,    0,    0],
       [  20,    0,    0,    3,   38,    0,    8,    0,    0,    1,    3,
           4,    0,   59,    0,    2,    1,    0],
       [   1,    0,    0,    0,    2,    0,    0,    0,    0,    2,    0,
           0,    0,    2,    0,    0,    0,    0],


In [44]:
print('e128 f1: city: {}, class: {}'.format(e128_city_f1, e128_class_f1))
print('e256 f1: city: {}, class: {}'.format(e256_city_f1, e256_class_f1))
print('e512 f1: city: {}, class: {}'.format(e512_city_f1, e512_class_f1))
print('final f1: city: {}, class: {}'.format(final_city_f1, final_class_f1))

e128 f1: city: 0.656499379322575, class: 0.39989359815570136
e256 f1: city: 0.6258201808831353, class: 0.29916651888632734
e512 f1: city: 0.5917715907075722, class: 0.4162085476148254
final f1: city: 0.7996098599042383, class: 0.4593012945557723


In [45]:
from sklearn.metrics import classification_report

target_names = ''

# classification_report(city_y, y_final_city_pred, target_names=target_names)
print(classification_report(city_y, y_e128_city_pred))
print(classification_report(city_y, y_e256_city_pred))
print(classification_report(city_y, y_e512_city_pred))
print(classification_report(city_y, y_final_city_pred))

              precision    recall  f1-score   support

           0       0.56      0.71      0.63       991
           1       0.80      0.73      0.76      1126
           2       0.94      0.44      0.60       980
           3       0.62      0.86      0.72      1527
           4       0.55      0.42      0.48      1015

    accuracy                           0.66      5639
   macro avg       0.69      0.63      0.64      5639
weighted avg       0.69      0.66      0.65      5639

              precision    recall  f1-score   support

           0       0.59      0.51      0.55       991
           1       0.80      0.82      0.81      1126
           2       0.85      0.34      0.48       980
           3       0.52      0.95      0.67      1527
           4       0.69      0.31      0.43      1015

    accuracy                           0.63      5639
   macro avg       0.69      0.59      0.59      5639
weighted avg       0.68      0.63      0.60      5639

              precisio

In [46]:
print(classification_report(class_y, y_e128_class_pred))
print(classification_report(class_y, y_e256_class_pred))
print(classification_report(class_y, y_e512_class_pred))
print(classification_report(class_y, y_final_class_pred))

              precision    recall  f1-score   support

           0       0.52      0.69      0.59      1681
           1       0.00      0.00      0.00        35
           2       0.00      0.00      0.00        28
           3       0.21      0.41      0.28       332
           4       0.14      0.07      0.10       391
           5       0.08      0.07      0.08        55
           6       0.08      0.02      0.03       139
           7       0.00      0.00      0.00         7
           8       0.00      0.00      0.00        54
           9       0.44      0.36      0.39       225
          10       0.27      0.09      0.14       171
          11       0.22      0.07      0.10       338
          12       0.00      0.00      0.00         3
          13       0.41      0.41      0.41      1870
          14       0.10      0.03      0.05        30
          15       0.00      0.00      0.00        91
          16       0.06      0.01      0.02        89
          17       0.26    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classificat